In [5]:
import numpy as np
import cv2
from scipy.stats import itemfreq
import mahotas as mt

def extract_features(image):
	# calculate haralick texture features for 4 types of adjacency
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    textures = mt.features.haralick(gray)
	# take the mean of it and return it
    ht_mean  = textures.mean(axis=0)
    arr = np.float32(image)
    pixels = arr.reshape((-1, 3))
    n_colors = 3
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS
    _, labels, centroids = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
    palette = np.uint8(centroids)
    quantized = palette[labels.flatten()]
    quantized = quantized.reshape(image.shape)
    dominant_color = palette[np.argmax(itemfreq(labels)[:, -1])]
    feat = np.concatenate((ht_mean,dominant_color))
    return feat

ImportError: cannot import name 'itemfreq' from 'scipy.stats' (C:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\__init__.py)

In [4]:
import cv2
import numpy as np
import os
import glob
from sklearn import tree
from fruit_feature import extract_features

# load the training dataset
train_path  = "ap-or-database/train"
train_names = os.listdir(train_path)

# empty list to hold feature vectors and train labels
train_features = []
train_labels   = []

# loop over the training dataset
for train_name in train_names:
	cur_path = train_path + "/" + train_name
	cur_label = train_name
	i = 1

	for file in glob.glob(cur_path + "/*.jpg"):
		print ("Processing Image - {} in {}".format(i, cur_label))
		# read the training image
		image = cv2.imread(file)

		# extract texture and color from the image
		features = extract_features(image)

		# append the feature vector and label
		train_features.append(features)
		train_labels.append(cur_label)

		# show loop update
		i += 1


# create the classifier
clf=tree.DecisionTreeClassifier()

# train the classifier
print ("Training model..")
clf.fit(train_features, train_labels)

# loop over the test images
test_path = ("ap-or-database/test")
for file in glob.glob(test_path + "/*.jpg"):
	# read the input image
    image = cv2.imread(file)

		# extract texture and color from the image
    features = extract_features(image)

	# evaluate the model and predict label
    prediction = clf.predict(features.reshape(1, -1))[0]

	# show the label
    cv2.putText(image, prediction, (20,30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (235,188,0), 3)
    print ("Prediction - {}".format(prediction))

	# display the output image
    cv2.imshow("Test_Image", image)
    cv2.waitKey(0)
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'fruit_feature'